In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/disease-symptom-description-dataset/symptom_Description.csv
/kaggle/input/disease-symptom-description-dataset/Symptom-severity.csv
/kaggle/input/disease-symptom-description-dataset/symptom_precaution.csv
/kaggle/input/disease-symptom-description-dataset/dataset.csv


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import networkx as nx

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Load the datasets
symptom_severity_df = pd.read_csv('/kaggle/input/disease-symptom-description-dataset/Symptom-severity.csv')
disease_symptom_df = pd.read_csv('/kaggle/input/disease-symptom-description-dataset/dataset.csv')
disease_description_df = pd.read_csv('/kaggle/input/disease-symptom-description-dataset/symptom_Description.csv')
disease_precaution_df = pd.read_csv('/kaggle/input/disease-symptom-description-dataset/symptom_precaution.csv')

In [4]:
# Incorporate symptom severity
symptom_severity_dict = dict(zip(symptom_severity_df['Symptom'], symptom_severity_df['weight']))

# Create a dictionary to map symptoms to their indices
symptom_to_index = {symptom: index for index, symptom in enumerate(symptom_severity_dict.keys())}

# Remove leading and trailing spaces from disease names in the disease_symptom_df dataset
disease_symptom_df['Disease'] = disease_symptom_df['Disease'].str.strip()
disease_symptom_df = disease_symptom_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Remove leading and trailing spaces from disease names in the disease_description_df dataset
disease_description_df['Disease'] = disease_description_df['Disease'].str.strip()

In [5]:
# Define symbolic rules
disease_symptom_dict = {}
symptom_disease_dict = {}

# Normalize symptom names
normalize_symptom = lambda symptom: symptom.strip().lower()

# Iterate over the rows of the dataset
for _, row in disease_symptom_df.iterrows():
    disease = row['Disease']
    symptoms = [normalize_symptom(symptom) for symptom in row.drop('Disease').dropna().tolist()]
    
    if disease in disease_symptom_dict:
        disease_symptom_dict[disease].extend(symptoms)
    else:
        disease_symptom_dict[disease] = symptoms
    
    for symptom in symptoms:
        if symptom in symptom_disease_dict:
            symptom_disease_dict[symptom].append(disease)
        else:
            symptom_disease_dict[symptom] = [disease]

# Convert patient symptoms to a one-hot encoded vector
def encode_symptoms(symptoms):
    encoded = np.zeros(len(symptom_severity_dict))
    for symptom in symptoms:
        symptom = normalize_symptom(symptom)
        if symptom in symptom_severity_dict and symptom in symptom_to_index:
            severity = symptom_severity_dict[symptom]
            encoded[symptom_to_index[symptom]] = severity
    return encoded

# Create the hybrid model
symptom_input = layers.Input(shape=(len(symptom_severity_dict),), name='symptom_input')

# Neural network component
neural_network_output = layers.Dense(256, activation='relu')(symptom_input)
neural_network_output = layers.Dropout(0.5)(neural_network_output)
neural_network_output = layers.Dense(128, activation='relu')(neural_network_output)
neural_network_output = layers.Dropout(0.5)(neural_network_output)
neural_network_output = layers.Dense(64, activation='relu')(neural_network_output)
neural_network_output = layers.Dropout(0.5)(neural_network_output)
neural_network_output = layers.Dense(32, activation='relu')(neural_network_output)
neural_network_output = layers.Dropout(0.5)(neural_network_output)
neural_network_output = layers.Dense(len(disease_symptom_dict), activation='softmax', name='neural_network_output')(neural_network_output)

# Symbolic reasoning component
# Create the hybrid model
symbolic_rules_output = layers.Dense(len(disease_symptom_dict), activation='softmax', name='symbolic_rules_output')(symptom_input)

# Combine neural network and symbolic reasoning outputs using an average layer
combined_output = layers.Average(name='combined_output')([neural_network_output, symbolic_rules_output])

# Create the hybrid model
model = Model(inputs=[symptom_input], outputs=[combined_output])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert disease-symptom dictionary to training data
X_train = []
y_train = []
for disease, symptoms in disease_symptom_dict.items():
    X_train.append(encode_symptoms(symptoms))
    y_train.append(np.array([1 if d == disease else 0 for d in disease_symptom_dict.keys()]))
X_train = np.array(X_train)
y_train = np.array(y_train)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16)

Epoch 1/100
3/3 [==============================] - 1s 7ms/step - loss: 3.8289 - accuracy: 0.0244
Epoch 2/100
3/3 [==============================] - 0s 5ms/step - loss: 3.7527 - accuracy: 0.0488
Epoch 3/100
3/3 [==============================] - 0s 5ms/step - loss: 3.6902 - accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 4ms/step - loss: 3.6229 - accuracy: 0.0488
Epoch 5/100
3/3 [==============================] - 0s 4ms/step - loss: 3.4199 - accuracy: 0.0488
Epoch 6/100
3/3 [==============================] - 0s 4ms/step - loss: 3.5333 - accuracy: 0.0244
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 3.3509 - accuracy: 0.0976
Epoch 8/100
3/3 [==============================] - 0s 4ms/step - loss: 3.4009 - accuracy: 0.1463
Epoch 9/100
3/3 [==============================] - 0s 5ms/step - loss: 3.3124 - accuracy: 0.1220
Epoch 10/100
3/3 [==============================] - 0s 5ms/step - loss: 3.2522 - accuracy: 0.1463
Epoch 11/100
3/3 [=======

In [6]:
# Function to get feature importance based on model weights
def get_feature_importance(model, user_symptoms):
    feature_names = list(symptom_severity_dict.keys())
    user_symptom_indices = np.array([symptom_to_index[symptom] for symptom in user_symptoms if symptom in symptom_to_index])
    feature_importance = np.abs(model.get_weights()[0]).sum(axis=1)
    user_symptom_importance = feature_importance[user_symptom_indices]
    sorted_indices = np.argsort(user_symptom_importance)[::-1]
    sorted_features = [feature_names[i] for i in user_symptom_indices[sorted_indices]]
    sorted_importance = user_symptom_importance[sorted_indices]
    return sorted_features, sorted_importance


# Function to predict disease based on symptoms using the trained model
def predict_disease(symptoms):
    encoded_symptoms = encode_symptoms(symptoms)
    predicted = model.predict(np.array([encoded_symptoms]))
    disease_index = np.argmax(predicted)
    predicted_disease = list(disease_symptom_dict.keys())[disease_index]

    # Add explanation within the prediction process
    print("The model predicted the disease based on the following symptoms:")
    for symptom in symptoms:
        print("- " + symptom)
    print("The predicted disease is: " + predicted_disease)

    return predicted_disease


def explain_decision(symptoms, user_symptoms):
    print("symptoms", symptoms)
    encoded_symptoms = encode_symptoms(symptoms)
    predicted = model.predict(np.array([encoded_symptoms]))
    disease_index = np.argmax(predicted)
    predicted_disease = list(disease_symptom_dict.keys())[disease_index]
    top_diseases_indices = np.argsort(predicted)[0][-5:][::-1]
    top_diseases = [list(disease_symptom_dict.keys())[index] for index in top_diseases_indices]
    
    # Get disease description and precautions
    disease_description = get_disease_description(predicted_disease)
    disease_precautions = get_disease_precautions(predicted_disease)
    
    # Check for other diseases with the same symptoms
    potential_diseases = []
    for disease, symptoms in disease_symptom_dict.items():
        if set(user_symptoms).issubset(set(symptoms)):
            potential_diseases.append(disease)
    print("POTENTIAL", potential_diseases)
    
    diseases_same = []
    for disease in potential_diseases:
        dsymptoms = disease_symptom_dict[disease]
        check = all(item in dsymptoms for item in user_symptoms)
        if check is True:
            print()
            diseases_same.append(disease)
            print()
    
    print("TOP", top_diseases)
    for i in top_diseases:
        diseases_same.append(i)
    
    diseases_same = set(diseases_same)
    diseases_same = list(diseases_same)
    count = len(diseases_same)
    
    print()
    print("Total diseases with these symptoms", count)
    for i in diseases_same:
        print(i)
    print()
    
    check = input("Do you wanna check for other diseases?")
    if check == 'yes':
        for i in diseases_same:
            print("DISEASE", i)
            print("symptoms", disease_symptom_dict[i])
            dsd = set(disease_symptom_dict[i])
            print("user symptoms", user_symptoms)
            us = set(user_symptoms)
            uncommon_symptoms = list(dsd.difference(us))
            print("uncommon symptoms", uncommon_symptoms)
            for j in uncommon_symptoms[:2]:
                print("What about this?", j)
                ans = input("do u have this? (yes or no)")
                if ans == 'yes':
                    user_symptoms.append(j)
    else:
        print("Hopefully we got that right ;)")
        
    encoded_symptoms = encode_symptoms(user_symptoms)
    predicted = model.predict(np.array([encoded_symptoms]))
    disease_index = np.argmax(predicted)
    predicted_disease = list(disease_symptom_dict.keys())[disease_index]

    # Get disease description and precautions
    disease_description = get_disease_description(predicted_disease)
    disease_precautions = get_disease_precautions(predicted_disease)
    
    # Generate detailed explanation
    explanation = f"Based on the given symptoms, the model predicts the disease as '{predicted_disease}'.\n\n"

    # Feature Importance
    explanation += "Feature Importance:\n"
    features, importance = get_feature_importance(model, user_symptoms)
    explanation += "The most important symptoms for the prediction are:\n"
    for feature, score in zip(features, importance):
        explanation += f"- {feature}: Importance {score}\n"
    explanation += "\n"

    # Neurosymbolic Explanation
    explanation += "Neurosymbolic Explanation:\n"
    explanation += "The model combines neural network predictions and symbolic reasoning to arrive at the disease prediction. "
    explanation += "The neural network analyzes the severity of symptoms and learns patterns from the training data. "
    explanation += "The symbolic reasoning component uses predefined rules and relationships between symptoms and diseases to infer the most likely disease.\n\n"

    # Symptom Analysis
    explanation += "Symptom Analysis:\n"
    explanation += "The model considers the following symptoms and their severities:\n"
    for symptom, severity in zip(symptom_severity_dict.keys(), encoded_symptoms):
        if symptom in user_symptoms:
            explanation += f"- {symptom}: Severity {severity}\n"
    explanation += "\n"

    # Disease-Symptom Information
    explanation += "Disease-Symptom Information:\n"
    explanation += "The model incorporates information about diseases and their associated symptoms from the dataset.\n"
    explanation += "It learns the relationships between symptoms and diseases through the training process.\n"
    disease_symptoms = disease_symptom_dict[predicted_disease]

    # Disease Description
    explanation += "Description:\n"
    explanation += f"The predicted disease, '{predicted_disease}', is described as follows:\n"
    explanation += f"{disease_description}\n\n"

    # Precautions
    explanation += "Precautions:\n"
    for i, precaution in enumerate(disease_precautions):
        explanation += f"{i + 1}. {precaution}\n"

    return explanation

def get_disease_description(disease):
    description = disease_description_df.loc[disease_description_df['Disease'] == disease, 'Description'].values[0]
    return description

# Function to get disease precautions
def get_disease_precautions(disease):
    precautions = disease_precaution_df.loc[disease_precaution_df['Disease'] == disease].drop('Disease', axis=1).values[0]
    return precautions

In [7]:
# Example usage
patient_symptoms = ['itching','skin_rash']
predicted_disease = predict_disease(patient_symptoms)
explanation = explain_decision(patient_symptoms, patient_symptoms)
print('Patient Symptoms:', patient_symptoms)
print('Predicted Disease:', predicted_disease)
print('Explanation:\n', explanation)
print('Attention Visualization')
sorted_features, sorted_importance = get_feature_importance(model, patient_symptoms)
print('Feature Importance:')
for feature, importance in zip(sorted_features, sorted_importance):
    print(f'- {feature}: {importance}')

1/1 [==============================] - 0s 99ms/step
The model predicted the disease based on the following symptoms:
- itching
- skin_rash
The predicted disease is: Drug Reaction
symptoms ['itching', 'skin_rash']
1/1 [==============================] - 0s 18ms/step
POTENTIAL ['Fungal infection', 'Drug Reaction', 'Chicken pox']






TOP ['Drug Reaction', 'Fungal infection', 'Acne', 'Chicken pox', 'Gastroenteritis']

Total diseases with these symptoms 5
Fungal infection
Drug Reaction
Chicken pox
Acne
Gastroenteritis



Do you wanna check for other diseases? No


Hopefully we got that right ;)
1/1 [==============================] - 0s 18ms/step
Patient Symptoms: ['itching', 'skin_rash']
Predicted Disease: Drug Reaction
Explanation:
 Based on the given symptoms, the model predicts the disease as 'Drug Reaction'.

Feature Importance:
The most important symptoms for the prediction are:
- skin_rash: Importance 16.518003463745117
- itching: Importance 15.642765998840332

Neurosymbolic Explanation:
The model combines neural network predictions and symbolic reasoning to arrive at the disease prediction. The neural network analyzes the severity of symptoms and learns patterns from the training data. The symbolic reasoning component uses predefined rules and relationships between symptoms and diseases to infer the most likely disease.

Symptom Analysis:
The model considers the following symptoms and their severities:
- itching: Severity 1.0
- skin_rash: Severity 3.0

Disease-Symptom Information:
The model incorporates information about diseases and thei